In [ ]:
# Table: repo_collection

import psycopg2
import pandas as pd

# Database connection details
DB_NAME = "github_repos"
DB_USER = "postgres"
DB_PASSWORD = "Sphings@19"
DB_HOST = "localhost"
DB_PORT = "5432"

# CSV file path
CSV_FILE = "../input_files/repo_collections_list.csv"  # Update with actual file path

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()

    # Read CSV file
    df = pd.read_csv(CSV_FILE)  # Assuming tab-separated file
    df.columns = df.columns.str.strip()

    # Rename columns to match the table
    df.rename(columns={
        "Organization": "organization",
        "html_url": "html_url",
        "fork": "fork",
        "created_at": "created_at",
        "updated_at": "updated_at",
        "pushed_at": "pushed_at",
        "git_url": "git_url",
        "size": "size",
        "stargazers_count": "stargazers_count",
        "watchers_count": "watchers_count",
        "language": "language",
        "forks_count": "forks_count",
        "archived": "archived",
        "disabled": "disabled",
        "open_issues_count": "open_issues_count",
        "license": "license",
        "allow_forking": "allow_forking"
    }, inplace=True)

    # Replace NaN with None for PostgreSQL compatibility
    df = df.where(pd.notna(df), None)

    # Insert data
    for _, row in df.iterrows():
        cur.execute("""
            INSERT INTO repo_collection (
                organization, html_url, fork, created_at, updated_at, pushed_at,
                git_url, size, stargazers_count, watchers_count, language,
                forks_count, archived, disabled, open_issues_count, license, allow_forking
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """, (
            row["organization"], row["html_url"], row["fork"], row["created_at"],
            row["updated_at"], row["pushed_at"], row["git_url"], row["size"],
            row["stargazers_count"], row["watchers_count"], row["language"],
            row["forks_count"], row["archived"], row["disabled"],
            row["open_issues_count"], row["license"], row["allow_forking"]
        ))

    # Commit and close
    conn.commit()
    print("Data inserted successfully.")

except Exception as e:
    print("Error:", e)

finally:
    if conn:
        cur.close()
        conn.close()


Data inserted successfully.


In [ ]:
# Table: searchrepos
import os
import psycopg2
import pandas as pd
from datetime import datetime

# Database connection details
DB_NAME = "github_repos"
DB_USER = "postgres"
DB_PASSWORD = "Sphings@19"
DB_HOST = "localhost"
DB_PORT = "5432"

# CSV file path
CSV_FILE = "../input_files/repo_collections_list.csv" 

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()

    # OPTIONAL: Empty the table before inserting data (comment this if not needed)
    cur.execute("TRUNCATE TABLE searchrepos RESTART IDENTITY;")
    conn.commit()
    print("Table cleared.")

    # Read CSV file
    df = pd.read_csv(CSV_FILE)  # Adjust delimiter if needed

    # Clean column names (strip spaces)
    df.columns = df.columns.str.strip()

    # Rename columns to match the database
    df.rename(columns={
        "Organization": "organization",
        "html_url": "repository_url",
        "license": "license",
        "language": "language",
    }, inplace=True)

    # Extract project_id from repository_url
    df["project_id"] = ""

    # Generate unique ID (_id) using organization + project_id
    df["_id"] = df.apply(lambda row: datetime.now().strftime("%Y%m%d%H%M%S%f"), axis=1)

    # Fill missing values with defaults
    df["licenseConflicts"] = 0  # Default to 0
    df["is_active"] = True  # Default to True

    # Replace NaN with None for PostgreSQL
    df = df.where(pd.notna(df), None)

    # Insert data
    for _, row in df.iterrows():
        cur.execute("""
            INSERT INTO searchrepos (
                _id, organization, project_id, repository_url, license, language, 
                licenseConflicts, is_active
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (_id) DO NOTHING;
        """, (
            row["_id"], row["organization"], row["project_id"], row["repository_url"],
            row["license"], row["language"], row["licenseConflicts"], row["is_active"]
        ))

    # Commit changes
    conn.commit()
    print("Data inserted successfully.")

except Exception as e:
    print("Error:", e)

finally:
    if conn:
        cur.close()
        conn.close()

Table cleared.
Data inserted successfully.
